In [1]:

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import folium
from folium.map import Element
import seaborn as sns
from IPython.display import Markdown as md
from IPython.display import display, HTML

import xyzservices.providers as xyz


In [2]:
zips = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2022/ZCTA520/tl_2022_us_zcta520.zip")
zips

HTTPError: HTTP Error 403: Forbidden

In [3]:
df = schools.load_school_demographics()
df = df[df.ay == df.ay.max()]
df["district"] = df["dbn"].apply(lambda dbn: int(dbn[:2]))

boros = {"K":"Brooklyn", "X":"Bronx", "M": "Manhattan", "Q": "Queens", "R": "Staten Island"}
df["boro"] = df["dbn"].apply(lambda dbn: boros[dbn[2]])
df["school_num"] = df.dbn.apply(lambda dbn: int(dbn[3:]))
df["charter"] = df.district.apply(lambda x: 1 if x == 84 else 0)

# figure out what grades they teach
df["pk"] = df["grade_pk"] > 0
df["elementary"] = df["grade_2"] > 0
df["middle"] = df["grade_7"] > 0
df["hs"] = df["grade_10"] > 0
df

dbn          beds  district  geo_district       boro  \
4     01M015  310100010015         1             1  Manhattan   
9     01M020  310100010020         1             1  Manhattan   
14    01M034  310100010034         1             1  Manhattan   
19    01M063  310100010063         1             1  Manhattan   
24    01M064  310100010064         1             1  Manhattan   
...      ...           ...       ...           ...        ...   
9342  84X705  320900860839        84             9      Bronx   
9347  84X706  320900860823        84             9      Bronx   
9352  84X717  320900860835        84             9      Bronx   
9357  84X718  321100860855        84            11      Bronx   
9362  84X730  320800860846        84             8      Bronx   

                                   school_name short_name    ay     year  \
4                    P.S. 015 Roberto Clemente      PS 15  2023  2023-24   
9                         P.S. 020 Anna Silver      PS 20  2023  2023-24   
14              P.S. 034 Franklin D. Roosevelt      PS 34  2023  2023-24   
19                   The STAR Academy - P.S.63      PS 63  2023  2023-24   
24                       P.S. 064 Robert Simon      PS 64  2023  2023-24   
...                                        ...        ...   ...      ...   
9342        Family Life Academy Charter School     PS 705  2023  2023-24   
9347             Harriet Tubman Charter School     MS 706  2023  2023-24   
9352                    Icahn Charter School 1     MS 717  2023  2023-24   
9357  Bronx Charter School for Better Learning     PS 718  2023  2023-24   
9362         Bronx Charter School for the Arts     MS 730  2023  2023-24   

     school_type  ...  missing_race_ethnicity_data_pct  swd_n   swd_pct  \
4      community  ...                         0.000000   44.0  0.232804   
9      community  ...                         0.000000   63.0  0.165789   
14     community  ...                         0.000000   57.0  0.262673   
19     community  ...                         0.000000   64.0  0.310680   
24     community  ...                         0.005376   56.0  0.301075   
...          ...  ...                              ...    ...       ...   
9342     charter  ...                         0.002445   50.0  0.122249   
9347     charter  ...                         0.002037   57.0  0.116090   
9352     charter  ...                         0.000000   26.0  0.077151   
9357     charter  ...                         0.003636   54.0  0.098182   
9362     charter  ...                         0.000000  130.0  0.208333   

      ell_n   ell_pct  poverty_n  poverty_pct       eni  \
4      24.0  0.126984        163     0.862434  0.875365   
9     119.0  0.313158        326     0.857895  0.855750   
14     39.0  0.179724        209     0.960000  0.949327   
19     31.0  0.150485        164     0.796117  0.801063   
24     25.0  0.134409        168     0.903226  0.896446   
...     ...       ...        ...          ...       ...   
9342  100.0  0.244499        393     0.960000  0.960000   
9347   63.0  0.128310        431     0.877800  0.932961   
9352   13.0  0.038576        309     0.916914  0.900442   
9357    6.0  0.010909        464     0.843636  0.722634   
9362   70.0  0.112179        554     0.887821  0.915459   

                                    clean_name    zip  
4                             roberto clemente  10009  
9                                  anna silver  10002  
14                        franklin d roosevelt  10009  
19                     the star academy - ps63  10009  
24                                robert simon  10009  
...                                        ...    ...  
9342        family life academy charter school  10452  
9347             harriet tubman charter school  10456  
9352                    icahn charter school 1  10457  
9357  bronx charter school for better learning  10466  
9362         bronx charter school for the arts  10474  

[1892 rows x 55 columns]

In [21]:
pz = geo.load_zipcodes()
# zips["zip"] = zips.ZCTA5CE20.astype(int)
pz["zip"] = pz.zip.astype(int)
# sz = zips[zips.zip.isin(df.zip.unique())]
# df.zip
df[~df.zip.isin(pz.zip) & df.zip > 0][["dbn", "zip"]]


,dbn,zip


In [5]:
# most recent URL
url = "https://data.cityofnewyork.us/download/jfju-ynrr/application%2Fx-zip-compressed"
points = gpd.read_file(url)
display(points.columns)
points

Index(['ATS', 'Building_C', 'Location_C', 'Name', 'Geographic', 'Latitude',
       'Longitude', 'geometry'],
      dtype='object')

,ATS,Building_C,Location_C,Name,Geographic,Latitude,Longitude,geometry
0,01M015,M015,M015,P.S. 015 Roberto Clemente,1,40.722075,-73.978747,POINT (-8235276.446 4971433.816)
1,01M020,M020,M020,P.S. 020 Anna Silver,1,40.721305,-73.986312,POINT (-8236118.578 4971320.718)
2,01M034,M034,M034,P.S. 034 Franklin D. Roosevelt,1,40.726008,-73.975058,POINT (-8234865.788 4972011.521)
3,01M063,M063,M063,The STAR Academy - P.S.63,1,40.724440,-73.986214,POINT (-8236107.668 4971781.199)
4,01M064,M064,M064,P.S. 064 Robert Simon,1,40.723130,-73.981597,POINT (-8235593.706 4971588.778)
...,...,...,...,...,...,...,...,...
1945,88X966,X988,X966,ALC - BRONX EDUCATIONAL CENTER,8,40.816494,-73.890278,POINT (-8225428.122 4985312.134)
1946,88X994,X358,X994,ALC - Crotona Academy,12,40.829306,-73.892243,POINT (-8225646.864 4987196.846)
1947,88X995,X896,X995,ALC - Rose Hill Academy,10,40.857248,-73.903165,POINT (-8226862.696 4991308.524)
1948,88X996,X450,X996,ALC - Stevenson Campus,8,40.821218,-73.855930,POINT (-8221604.52 4986007.017)


In [6]:
cols = {
    'ATS':"dbn", 
    'Geographic':"geo_district", 
    'geometry':'geometry'
}
points.rename(columns=cols, inplace=True)
points = points[cols.values()]
points

,dbn,geo_district,geometry
0,01M015,1,POINT (-8235276.446 4971433.816)
1,01M020,1,POINT (-8236118.578 4971320.718)
2,01M034,1,POINT (-8234865.788 4972011.521)
3,01M063,1,POINT (-8236107.668 4971781.199)
4,01M064,1,POINT (-8235593.706 4971588.778)
...,...,...,...
1945,88X966,8,POINT (-8225428.122 4985312.134)
1946,88X994,12,POINT (-8225646.864 4987196.846)
1947,88X995,10,POINT (-8226862.696 4991308.524)
1948,88X996,8,POINT (-8221604.52 4986007.017)


In [ ]:
zips = geo.load_zipcodes()
points.to_crs(zips.crs, inplace=True)
pz = gpd.sjoin( points, zips, how='left', predicate='within')
pz.drop(columns=['index_right'], inplace=True)
pz["district"] = pz.dbn.apply(lambda x: int(x[0:2]))

pz


,dbn,geo_district,geometry,zip,district
0,01M015,1,POINT (-73.97875 40.72208),10009,1
1,01M020,1,POINT (-73.98631 40.72131),10002,1
2,01M034,1,POINT (-73.97506 40.72601),10009,1
3,01M063,1,POINT (-73.98621 40.72444),10009,1
4,01M064,1,POINT (-73.9816 40.72313),10009,1
...,...,...,...,...,...
1945,88X966,8,POINT (-73.89028 40.81649),10474,88
1946,88X994,12,POINT (-73.89224 40.82931),10459,88
1947,88X995,10,POINT (-73.90316 40.85725),10453,88
1948,88X996,8,POINT (-73.85593 40.82122),10473,88


In [27]:
op = geo.get_points()
op = op[op.dbn.isin(df.dbn)]
ol = geo.get_locations()
# url = "https://data.cityofnewyork.us/resource/wg9x-4ke6.csv?$limit=1000000"
# ol = pd.read_csv(url)
ol.columns


Index(['dbn', 'administrative_district_code', 'administrative_district_name',
       'beds', 'borough_block_lot', 'census_tract', 'community_district',
       'community_school_sup_name', 'council_district', 'fax_number',
       'fiscal_year', 'geographical_district_code', 'grades_final_text',
       'grades_text', 'highschool_network', 'highschool_network_location',
       'highschool_network_name', 'latitude', 'location_category_description',
       'location_code', 'location_name', 'location_type_description',
       'longitude', 'managed_by_name', 'nta', 'nta_name', 'open_year',
       'state_code', 'status_descriptions'],
      dtype='object')

In [28]:

ol["bbl"] = ol.borough_block_lot.apply(lambda x: str(x).split(".")[0])

# ol.bbl = ol.bbl.str.split(".")[0]
ol.bbl

0       3007550022
1       3050500051
2       3019980001
3       3044010001
4       3014900001
           ...    
2184    2052630070
2185    2022930078
2186    2027550125
2187    2025780027
2188    2030070008
Name: bbl, Length: 2131, dtype: object

In [41]:
foot = geo.load_school_footprints()
ol[["dbn","bbl","location_name"]][(~ol.bbl.isin(foot.bbl)) & (ol.dbn.isin(df.dbn)) & (ol.bbl != "0")]
# op[["dbn","bbl"]][~op.bbl.isin(foot.bbl)]

,dbn,bbl,location_name
263,22K278,3084960100,J.H.S. 278 Marine Park
296,13K313,3000367504,Dock Street School for STEAM Studies
461,15K529,3055870038,West Brooklyn Community High School
571,15K667,3006880026,Sunset Park High School
644,84K766,3001470044,The International Charter School of New York
...,...,...,...
2059,84X488,2051350200,The Equality Charter School
2060,84X489,2026090002,South Bronx Classical Charter School III
2139,84X588,2026090002,South Bronx Classical Charter School IV
2156,84X613,2037107501,Bold Charter School


In [88]:
from nycschools import geo
from shapely import Point
old_loc = geo.load_school_geo_points()
loc.crs, old_loc.crs

(<Derived Projected CRS: EPSG:3857>
 Name: WGS 84 / Pseudo-Mercator
 Axis Info [cartesian]:
 - X[east]: Easting (metre)
 - Y[north]: Northing (metre)
 Area of Use:
 - name: World between 85.06°S and 85.06°N.
 - bounds: (-180.0, -85.06, 180.0, 85.06)
 Coordinate Operation:
 - name: Popular Visualisation Pseudo-Mercator
 - method: Popular Visualisation Pseudo Mercator
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich,
 <Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich)

In [ ]:
a = loc[["dbn", "geometry"]].copy()
b = old_loc[["dbn", "geometry"]].copy()
# reproject both to NYS plane
a = a.to_crs(epsg=2263)
b = b.to_crs(epsg=2263)

common = a[["dbn"]].merge(b[["dbn"]], how="inner")
a = a[a.dbn.isin(common.dbn)]
b = b[b.dbn.isin(common.dbn)]

a["xy"] = a.geometry.apply(lambda x: f"{round(x.x, 3),round(x.y, 3)}")
b["xy"] = b.geometry.apply(lambda x: f"{round(x.x, 3),round(x.y, 3)}")
union = a.merge(b[["dbn", "xy"]], on=["dbn", "xy"], how="inner")
diffa = a[~a.dbn.isin(union.dbn)]
diffb = b[~b.dbn.isin(union.dbn)]
sym = diffa[["dbn", "geometry"]].merge(diffb[["dbn", "geometry"]], on="dbn", how="inner")
sym["d"] = sym.apply(lambda x: x.geometry_x.distance(x.geometry_y), axis=1)
print(sym.sort_values(by="d", ascending=False).d.head(5))

# epsg = 2263
# 56888.083715
# 39903.024511
# 39448.698698
# 35206.342356
# 24493.494065

# epsg = 3857
# 22854.006727
# 16020.279858
# 15881.068213
# 14174.309343
# 9864.905424

1704    22854.006727
1743    16020.279858
1810    15881.068213
2027    14174.309343
1771     9864.905424
Name: d, dtype: float64


In [33]:
# schools not in loc
display("Schools not in location")
not_found = all_schools[ (~all_schools.DBN.isin(loc.dbn)) & (all_schools.Year == '2023-24')]
display(not_found)

# those 10 schools will not be in our data set, we will have to look up their point based on address
df = loc.merge(df, on="dbn", how="inner")

'Schools not in location'

,DBN,School Name,Year,Total Enrollment,Grade 3K,Grade PK (Half Day & Full Day),Grade K,Grade 1,Grade 2,Grade 3,...,% White,# Missing Race/Ethnicity Data,% Missing Race/Ethnicity Data,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty,Economic Need Index
725,02M933,City Knoll Middle School,2023-24,171,0,0,0,0,0,0,...,0.040936,0,0.000000,44,0.257310,76,0.444444,158,0.923977,0.936971
845,03M299,The Maxine Greene HS for Imaginative Inquiry,2023-24,93,0,0,0,0,0,0,...,0.075269,1,0.010753,31,0.333333,14,0.150538,85,0.913979,0.901
2172,09X404,School for Excellence,2023-24,103,0,0,0,0,0,0,...,0.009709,0,0.000000,47,0.456311,23,0.223301,96,0.932039,Above 95%
2567,10X382,"Elementary School for Math, Science, and Techn...",2023-24,226,0,0,30,41,27,42,...,0.000000,0,0.000000,59,0.261062,72,0.318584,Above 95%,Above 95%,Above 95%
2832,11X253,Bronx High School for Writing and Communicatio...,2023-24,203,0,0,0,0,0,0,...,0.039409,1,0.004926,52,0.256158,28,0.137931,Above 95%,Above 95%,0.90503
3022,12X098,J.H.S. 098 Herman Ridder,2023-24,110,0,0,0,0,0,0,...,0.009091,0,0.000000,22,0.200000,48,0.436364,Above 95%,Above 95%,0.9075
4145,17K531,"School for Human Rights, The",2023-24,143,0,0,0,0,0,0,...,0.034965,2,0.013986,43,0.300699,13,0.090909,124,0.867133,0.859755
4511,19K364,I.S. 364 Gateway,2023-24,130,0,0,0,0,0,0,...,0.007692,0,0.000000,46,0.353846,1,0.007692,116,0.892308,0.903669
4541,19K504,High School for Civil Rights,2023-24,204,0,0,0,0,0,0,...,0.009804,15,0.073529,49,0.240196,56,0.274510,178,0.872549,0.944
7873,75Q256,P.S. Q256,2023-24,553,3,5,38,50,34,34,...,0.099458,0,0.000000,512,0.925859,83,0.150090,492,0.889693,0.936984


In [34]:
# let's find all the colocated schools by matching schools with the same point
locations = pd.DataFrame()
# give each location an id
locations["geometry"] = df.geometry.unique()
locations["loc_id"] = locations.index + 1
# join the loc_id back into df using geometry
df = df.merge(locations, on="geometry", how="inner")

In [35]:
# create a dataframe for location campuses that have an ID a name, a total number of students, and number of schools
agg = {
    "dbn":"count",
    "total_enrollment":"sum",
    "school_name":"first", # this will be the largest school, b/c of how we sort the df
    "geometry":"first"
}
df.sort_values(by=["loc_id", "total_enrollment"], ascending=False, inplace=True)
df


,dbn,geo_district,geometry,school_name,year,total_enrollment,grade_3k,grade_pk,grade_k,grade_1,...,female_pct,nonbinary_pct,asian_pct,black_pct,hispanic_pct,white_pct,poverty_pct,swd_pct,ell_pct,loc_id
1881,84X730,8,POINT (-8225004.996 4986094.837),Bronx Charter School for the Arts,2023-24,624,0,0,32,51,...,0.580128,0.000000,0.003205,0.261218,0.713141,0.011218,0.887821,0.208333,0.112179,1349
1880,84X717,9,POINT (-8227033.905 4988572.215),Icahn Charter School 1,2023-24,337,0,0,35,41,...,0.554896,0.000000,0.023739,0.545994,0.409496,0.002967,0.916914,0.077151,0.038576,1348
1879,84X706,9,POINT (-8227152.794 4987641.451),Harriet Tubman Charter School,2023-24,491,0,0,49,52,...,0.503055,0.000000,0.000000,0.590631,0.378819,0.010183,0.877800,0.116090,0.128310,1347
1878,84X705,9,POINT (-8228588.148 4988808.820),Family Life Academy Charter School,2023-24,409,0,0,75,68,...,0.506112,0.000000,0.004890,0.158924,0.823961,0.000000,0.960000,0.122249,0.244499,1346
1877,84X703,9,POINT (-8226524.062 4988633.720),Bronx Preparatory Charter School,2023-24,895,0,0,0,0,...,0.531844,0.000000,0.005587,0.505028,0.472626,0.006704,0.850279,0.150838,0.051397,1345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,01M363,1,POINT (-8236107.668 4971781.199),Neighborhood School,2023-24,226,0,21,30,35,...,0.495575,0.004425,0.061947,0.110619,0.367257,0.362832,0.455752,0.269912,0.008850,4
3,01M063,1,POINT (-8236107.668 4971781.199),The STAR Academy - P.S.63,2023-24,206,14,15,32,29,...,0.495146,0.000000,0.009709,0.145631,0.689320,0.126214,0.796117,0.310680,0.150485,4
2,01M034,1,POINT (-8234865.788 4972011.521),P.S. 034 Franklin D. Roosevelt,2023-24,217,13,15,15,17,...,0.451613,0.000000,0.023041,0.373272,0.562212,0.032258,0.960000,0.262673,0.179724,3
1,01M020,1,POINT (-8236118.578 4971320.718),P.S. 020 Anna Silver,2023-24,380,0,28,47,56,...,0.510526,0.000000,0.157895,0.150000,0.581579,0.068421,0.857895,0.165789,0.313158,2


In [36]:
campuses = df[["loc_id", "dbn", "total_enrollment", "school_name", "geometry"]].groupby("loc_id").agg(agg).reset_index()
campuses.rename(columns={"dbn":"num_schools", "school_name":"campus"}, inplace=True)
campuses.sort_values(by="total_enrollment", ascending=False).head(50)

,loc_id,num_schools,total_enrollment,campus,geometry
446,447,1,5804,Brooklyn Technical High School,POINT (-8235019.075 4966561.627)
350,351,4,5175,The Bronx High School of Science,POINT (-8225287.080 4994651.588)
378,379,8,5168,Harry S Truman High School,POINT (-8218903.797 4993779.572)
885,886,1,4427,Francis Lewis High School,POINT (-8214827.278 4974195.591)
667,668,1,4001,Fort Hamilton High School,POINT (-8241883.925 4957453.579)
739,740,1,3890,Midwood High School,POINT (-8232338.613 4958333.947)
1132,1133,1,3802,Tottenville High School,POINT (-8259174.959 4942991.348)
740,741,1,3784,James Madison High School,POINT (-8231815.968 4955059.004)
704,705,1,3595,Edward R. Murrow High School,POINT (-8233093.916 4956404.047)
977,978,1,3432,Forest Hills High School,POINT (-8220273.361 4972600.424)


In [37]:
# "colocated" schools will be an school where their campus has > 1 school
df["colocated"] = df.loc_id.isin(campuses[campuses.num_schools > 1].loc_id)
df.colocated.value_counts()


False    1007
True      875
Name: colocated, dtype: int64

In [38]:
# just colocated schools
co = df[df.colocated == True]
co[pct_cols].describe()


,male_pct,female_pct,nonbinary_pct,asian_pct,black_pct,hispanic_pct,white_pct,poverty_pct,swd_pct,ell_pct
count,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000
mean,0.515805,0.483938,0.000257,0.069629,0.343067,0.480392,0.075514,0.834267,0.254873,0.162818
std,0.107178,0.107084,0.001297,0.108863,0.234832,0.235050,0.115897,0.153289,0.159152,0.165291
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.016260,0.000000,0.075000,0.000000,0.000000
25%,0.479076,0.450560,0.000000,0.010795,0.153765,0.276436,0.012983,0.801999,0.180912,0.056644
50%,0.513089,0.486747,0.000000,0.028889,0.290598,0.485870,0.026718,0.880519,0.232068,0.121495
75%,0.549061,0.520728,0.000000,0.079883,0.513339,0.664924,0.071351,0.936429,0.283514,0.206085
max,1.000000,1.000000,0.023256,0.929539,0.896725,1.000000,0.674757,0.960000,1.000000,1.000000


In [39]:
# just the solo schools
solo = df[df.colocated == False]
co[pct_cols].describe()

,male_pct,female_pct,nonbinary_pct,asian_pct,black_pct,hispanic_pct,white_pct,poverty_pct,swd_pct,ell_pct
count,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000
mean,0.515805,0.483938,0.000257,0.069629,0.343067,0.480392,0.075514,0.834267,0.254873,0.162818
std,0.107178,0.107084,0.001297,0.108863,0.234832,0.235050,0.115897,0.153289,0.159152,0.165291
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.016260,0.000000,0.075000,0.000000,0.000000
25%,0.479076,0.450560,0.000000,0.010795,0.153765,0.276436,0.012983,0.801999,0.180912,0.056644
50%,0.513089,0.486747,0.000000,0.028889,0.290598,0.485870,0.026718,0.880519,0.232068,0.121495
75%,0.549061,0.520728,0.000000,0.079883,0.513339,0.664924,0.071351,0.936429,0.283514,0.206085
max,1.000000,1.000000,0.023256,0.929539,0.896725,1.000000,0.674757,0.960000,1.000000,1.000000
